In [42]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models'

vectorfolder_bert = modelfolder + r'\BERT'
vectorfolder_ft   = modelfolder + r'\fasttext_wiki.hi'
vectorfolder_standford = modelfolder + r'\POS-Tagger-Hindi'

file           = datafolder_p + r'\1-Hinglish_SarcasmCSV.csv'
file_clean     = datafolder_p + r"\2-Hinglish_Sarcasm_Clean.csv"
file_FE        = datafolder_p + r"\3-features_pos.csv"

file_ft        = datafolder_p + r"\2-Hinglish_Sarcasm_Clean-fasttext.csv"
train_datafile = datafolder_p + r'\2-train.csv'
test_datafile  = datafolder_p + r'\2-test.csv'

In [43]:
#libraries required
import pandas as pd
import nltk
import re
import numpy as np

In [ ]:
nltk.download('punkt')

In [44]:
#Loadfiles in memory

#Load Hindi stopwords
df_stopwords = pd.read_csv(datafolder_e + "\Stopword_Hindi.csv")
stop_words= list(df_stopwords['words'])
stop_words=[] #don't do anything with stopwords

#Load 1-Hinglish_SarcasmCSV file
df= pd.read_csv(file, sep="\t")

In [45]:
def dev2engNumeric(txt):
    pat= list('०१२३४५६७८९')
    i=0
    for p in pat:
        txt=re.sub(p,str(i),txt)
        i+=1
    return txt

In [46]:
#remove following characters from text '[—\-`":“”~]'
def normalize_document(doc):

    #pattern = r'[.,;?।:-—”`~“]@#।'
    #pattern = r'[!.,?#।)()@:"]'
    pattern_sc = r'[—\-`":“”~]' #remplace these special characters with single space
    pattern_url = r'^(http|href|ftp|file)s?:\/\/.*[\r\n]*' #remove any url with null space
    pattern_at = r'@\S*' #remove any text after @ with null inlcuding @
    pattern_hash = '#\s?' #remove space after hashtag
    pattern_emo = '[\u210d-\U0001F9FF]' #emoticon identification
    
    # lower case and remove special characters\whitespaces. Not required for Devnagari Test
    #doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    
    #replace any character which has pattern with blank
    #print (doc, pattern_sc)
    doc = re.sub( pattern_sc,' ',doc)
    doc = re.sub( pattern_url,'',doc)
    doc = re.sub( pattern_at,'',doc)
    doc = re.sub( pattern_hash,'#',doc)
    
    #convert text to lower case. Not required for Devnagari Text
    #doc = doc.lower()
    #doc = doc.strip()
    
    #sometmes emoji and normal text is without space. To create a space between normal word and emoji. 
    #Otherwise it will tokenization problem.
    emo_pos=re.finditer(pattern_emo , doc)


    i=0
    j=0
    doc1=[]
    for pos in emo_pos:
        j=pos.end()
        if j<0:
            j=0
        doc1.append(doc[i:j-1]+" "+doc[j-1]+" " )

        i=pos.end()

    doc1.append( doc[j:]+" " )

        
    if len(doc1)>0: doc="".join(doc1) #use emoticon process text only if text contained any emoticon else leave.

    # tokenize document
    tokens = nltk.word_tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [47]:
#Clean the dataset

corpus = list(df.sentence)

normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)

df.sentence=norm_corpus

df['sentence'] = df.sentence.str.replace("‘","") #Regex was not able to remove these quotes
df['sentence'] = df.sentence.str.replace("’","") #Regex was not able to remove these quotes
df['sentence'] = df.sentence.str.replace("# ","#") #correct hastags format. It is not able to do in RE
df['sentence'] = df.sentence.str.replace(" & ","&") #remove spaces around &. J & K, A & N, Modi & Shah
df['sentence'] = df.sentence.str.replace("  "," ") #remove spaces around &. J & K, A & N, Modi & Shah
df['sentence'] = df.sentence.str.replace("।।",".") #this is fullstop marker in Devanagari
df['sentence'] = df.sentence.str.replace("।",".") #this is another fullstop marker in Devanagri
df['sentence'] = df.sentence.str.replace("|",".") #sometimes | is used as sentence marker
df['sentence'] = df.sentence.apply(dev2engNumeric) #replace devanagri number to english numbers
df['sentence'] = df['sentence'].str.replace(' +',' ') # remove any extra space
df['sentence_wo_emo'] = df['sentence'].str.replace(r'[\u210d-\U0001F9FF]','') # Remove emoticon. It does not help in POS tagging.
df['sentence_wo_emo'] = df['sentence_wo_emo'].str.replace(' +',' ')  # remove any extra space

In [ ]:
def tranlit2dev(doc, translit_hashtag=False):
    from indicnlp.transliterate import acronym_transliterator
    from indic_transliteration.sanscript import transliterate 
    from indic_transliteration import sanscript
    import re

    ack_transliterator=acronym_transliterator.LatinToIndicAcronymTransliterator()

    words=re.split(r' ',doc) #break sentence on space
    sent = []
    for w in words:
        
        #if first letter is not # then Translit is True.
        #if first Letter is # and tranlit_hastag=True then Translit is True
        #else Translit is false.
        Translit = (translit_hashtag and w[0]=='#') or (w[0]!='#')                                                      
                                                        
        
        if Translit:
            w1 = re.findall(r'[A-z]+',w) ##is  word Latin word?
            if len(w1)>0: #Found english word
                w2=w1[0]
                pat = re.compile(r'([A-Z][a-z]+)') #English word with camel case. Break camel case into different words
                w2= pat.split(w2)
                w4=[]
                for w3 in w2:
                    if len(w3)>0:    
                        if w3.upper()==w3: # all letters are capital in the word
                            w4.append( ack_transliterator.transliterate(w3,lang='hi') )
                        else:
                            w3=w3.lower()
                            w4.append( transliterate(w3, sanscript.ITRANS, sanscript.DEVANAGARI) )
                    else:
                        pass
                    #print (w4)
                w5="_".join(w4)

            else:
                w5=w
        else:
            w5=w

        sent.append(w5)
    return " ".join(sent)                                                                                                             

In [ ]:
docs = ['जेहादी यासिन Malik और आजाद Kashmir चिलाने वाले कश्मीर को बचाने आगे नहीं आएंगे #JammuKashmirFloods #KashmirFloods',
       'सरिया कानून के हिसाब से अब WIFE IS MOTHER',
       'अँधेरी रातों में सुनसान TL पर एक अश्लील लौंडा निकलता है उसे लोग Hawashmi कहते है',
       'DMK ने सरकार को आधा तलाक़ दिया SP&BSP ने सरकार से आधा विवाह किया पर चाहा कर भी Honeymoon नहीं मना सकते अपवित्र रिश्तों में ऐसा ही होता है .',
       'दिल्ली में शिला सरकार ने जो जनता पर बर्बरता से टियर गैस और डंडे बरसाए थे उसके पीछे कहीं वह 14 लाख फर्जी वोट तो प्रेरणा के श्रोत नहीं थे AAP',
       'अगर मुलायम और अखिलेश चाहें तो UP में एक दंगा और करवा के बिल महेंद्र सिंह धोनी के नाम पे फाड़ सकते हैं. #IndvsEng',
       'दिल बहुत ही हरामी कामिनी चीज़ होती है अच्छे खासे हस्ते खेलते आदमी को सेंटियापा का एडमिन बना देती है #SadStoriesOfTwitter',
       'NIA की गिरफ्त में तानिया परवीन हाफिज सईद को करती थी रिपोर्ट , हथियारों की ट्रेनिंग लेने जाना था पाकिस्तान',
      'मिग से F16 उड़ाऊँ ! चिड़िया से में बाज गिराऊँ ! फिर दुश्मन घर में चाय पी आऊँ ! मैं अभिनन्दन नाम कहाऊँ !',
      'और भी बहुत पत्रकारों पर F I R हुई थी वह तो तुमको नहीं दिखी अक्ल के अंधे नाम नयनसुख']

for doc in docs:
    print( tranlit2dev(doc, translit_hashtag=True) )
                                                                            

print ('='*20)                                                                            
for doc in docs:
    print( tranlit2dev(doc, translit_hashtag=False) )     

### <font color=red> Seeing above results of transliteration I am deciding not to do any transliteration</font>

In [48]:
df=df.set_index('ID')
df = df[['label','sentence','sentence_wo_emo']]

#save file for transliteration
df.to_csv(file_clean,sep="\t",encoding='utf-8')

# Experiment Area

In [ ]:
import unicodedata
def splitclusters(s):
    """Generate the grapheme clusters for the string s. (Not the full
    Unicode text segmentation algorithm, but probably good enough for
    Devanagari.)

    """
    virama = u'\N{DEVANAGARI SIGN VIRAMA}'
    cluster = u''
    last = None
    for c in s:
        cat = unicodedata.category(c)[0]
        if cat == 'M' or cat == 'L' and last == virama:
            cluster += c
        else:
            if cluster:
                yield cluster
            cluster = c
        last = c
    if cluster:
        yield cluster

In [ ]:
a="मेरा नाम हरी थप्लियाल है"
a="संयुक्त राज्य अमेरिका"
list(splitclusters(a))

In [ ]:
# re.sub('[—\-`":“”~]',' ','h~a  t—ha-pliyal”ri@pra`"s“a:d')
# re.sub('#[\s]*','$',"s # KashmirFloods")

# #re.sub(r'^(http|href|ftp|file)s?:\/\/.*[\r\n]*', '', 'file://pmlogy.com')


# #re.sub(, '', 'hari @prasad')

In [ ]:
# pattern_extraspace= "\s+"
# doc='💙ख़ूबसूरती हमेशा आपके 💙 ❣️चेहरे पर सजती रहे ❣️ 💙ख़ुशी हमेशा आपकी जिन्दगी 💙 ❣️ में महकती रहे।❣️ 💟 .'
# doc='कुछ ऐसे भी हादसे होते है जिंदगी में इंसान बच तो जाता है पर ज़िंदा नहीं रहता शुभ रात्रि मित्रों'
# #sometmes emoji and normal text is without space. To create a space between normal word and emoji. 
# #Otherwise it will tokenization problem.
# pattern_emo = '[\u210d-\U0001F900]'
# emo_pos=re.finditer(pattern_emo , doc)
# i=0
# doc1=[]
# for pos in emo_pos:
#     print (pos)
#     j=pos.end()-2
#     if j<0:
#         j=0
#     doc1.append(doc[i:j]+" " )
#     doc1.append( doc[pos.end()-1]+" " )
#     #print (j,txt1[i:j],txt1[b.end()-1])
#     i=pos.end()

# if len(doc1)>0: doc="".join(doc1)
# #doc = re.sub( pattern_extraspace,' ',doc) #remove extra space

# # tokenize document
# tokens = nltk.word_tokenize(doc)
# tokens

In [ ]:
import re
pattern_emo = '[\u210d-\U0001F9FF]'
#pattern_emo = '[\u210d-\U0001F1F3]'
#pattern_emo = '[\u1F1E6-\U0001F43B]'
#pattern_emo = '[\u1F600-\U0001F6FA]'
#[\u263a-\U0001f645]



#a=re.search('\u270d',a)
txt='तेरे चेहरे पर जो मक्कारी है उसे छिपाएगा कैसे मुल्क से गद्दारी तेरे खून में है उसे मिटाएगा कैसे ✍क्या करेगा \
जानकर अगर मुल्क के लिए कुछ 😀 करना ही था अच्छा तो पहले जा कर अब्दुल कलाम जी जैसा देश भक्त बनकर दिखता टुच्चे आदमी'
txt='इस उठाईगिरे आदमी की ऐसी ओछी हरकत के लिए ट्रेंड नही एफआईआर होनी चाहिए और दूसरी बात कल ही में प्रिंटिंग प्रेस पर जाकर इसके ट्वीट 👇का प्रिंटआउट निकलबा कर पूरे जिले के एक एक घर में बटवाउगा ताकि लोगों को पता चले कांग्रेस के नेता की सोच कितनी गिरी हुई और घटिया है #अरेस्ट_पंकज_पूनिया'
txt='मेरे देश के नौजवानों उठो और जल्दी Fb खोलो ललकियाँ 😍ऑनलाइन रही है। 😉😜'
txt='आज जो ट्वीट कर रही हूं वो ट्वीट अब तक का बेस्ट ट्वीट होने वाला है। अपने अपने एक देश भक्त हीरो का नाम 👇कमेंट बॉक्स में लिखे मेरे तो है चंद्रशेखर आजाद। और आपके ?'
txt='रहने दे मुझे #अँधेरे में ए ग़ालिब , 😒 उजालो में मुझे 🤗अपनों के #असली चेहरे नज़र 👀आ जाते है। ! 😓 #GoodMorningTwitte।'
#txt='फ्री की रोटी तो कुत्ते भी खाते हैं , भूख तो राष्ट्रवाद की होनी चाहिए । जयहिंद 🇮🇳 🚩🙏 🙏जय सनातन धर्म 🙏🏻'
#txt='🙋बेटी बीमार हो तो बड़ा दुख होता है बहू बीमार हो तो ड्रामा लगता है 😊😊'
#txt='OMG💙ख़ूबसूरती हमेशा आपके 💙 ❣️चेहरे पर सजती रहे ❣️ 💙ख़ुशी हमेशा आपकी जिन्दगी 💙 ❣️ में महकती रहे।❣️ 💟 . लगता है'
#txt='OMG😳ये है इस्लामी साँप मदरसों मे पढ़े हुए सपोले अगर मदरसे बंद नही हुए तो भारत मे आप के साथ यही होगा'
txt="सब गए ना 2019 में 👇 कोई छूटा तो नहीं 2018 में 😝 😅 😂 😆 #2018 #2019 #HappyNewYear2019 #HappyNewYear #नववर्षकीशुभकामनाएँ #नववर्ष"
txt1=txt
a=re.finditer(pattern_emo , txt)

i=0
txt2=[]
for b in a:
    j=b.end()
    if j<0:
        j=0
    print (j, txt1[i:j-1]+" "+txt1[j-1]+ " ")
    txt2.append(txt1[i:j-1]+" "+txt1[j-1]+ " " )
    #txt2.append( txt1[j:]+" " )

    i=b.end()

txt2.append( txt1[j:]+" " )
txt2="".join(txt2)
txt2

In [ ]:
txt1

In [ ]:
pat

In [ ]:


txt = ' इस्लामी १ साँप २ मदरसों ३ मे ४ पढ़े ५५ हुए ६७ सपोले अगर मदरसे बंद'
print( dev2engNumeric(txt) )